## 자연어 전처리-한글

In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_table('ratings_train.txt')[['document','label']]
test_data = pd.read_table('ratings_test.txt')[['document','label']]
data=pd.concat((train_data,test_data),axis=0)

In [3]:
data=data.rename(columns = {'label':'y', 'document' : 'X'})

In [4]:
s_w=set(['은','는','이','가','를','들','에게','의','을','도','으로','만','라서','하다'])
#s_w.add(불용어 추가문자열)
okt=Okt()

In [5]:
data.isnull().values.any()

True

In [6]:
data['X'].nunique(), data['y'].nunique()

(194543, 2)

In [7]:
data=data.drop_duplicates(subset=['X'])
data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') 
data['clean_X']=data.clean_X.str.replace('^ +','')
data['clean_X']=data.clean_X.replace('',np.nan) 
data=data.dropna(how='any') 

C:\Users\student\AppData\Local\Temp/ipykernel_13056/3039578657.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
C:\Users\student\AppData\Local\Temp/ipykernel_13056/3039578657.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.clean_X.str.replace('^ +','')


In [8]:
okt=Okt()
from tqdm import tqdm 
X_data=[] 
for i in tqdm(data['clean_X']): 
    tk_d=okt.morphs(i) 
    end_d=[w for w in tk_d if not w in s_w] 
    X_data.append(' '.join(end_d))

100%|█████████████████████████████████████████████████████████████████████████| 193518/193518 [05:06<00:00, 631.22it/s]


In [9]:
data['y_name']=data['y']
data['encoder_y']=LabelEncoder().fit_transform(data['y'])
data['categorical_y']=list(to_categorical(data['encoder_y']))

정규표현식 정리

In [10]:
Y=data['encoder_y']
#Y=to_categorical(data['encoder_y'])

In [11]:
X=X_data

In [12]:
x_data,test_x,y_data,test_y = train_test_split(X,Y,test_size=0.3,random_state=0)

In [13]:
train_x,val_x,train_y,val_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0)

In [14]:
tk = Tokenizer() 
tk.fit_on_texts(train_x)

In [15]:
n=len([d for d in sorted(list(tk.word_counts.items()),key=lambda x:x[1]) if d[1]>4])+1

In [16]:
token=Tokenizer(n)
token.fit_on_texts(train_x)

In [17]:
token_train_x=token.texts_to_sequences(train_x)
token_test_x=token.texts_to_sequences(test_x)
token_val_x=token.texts_to_sequences(val_x)

In [18]:
train_inputs = pad_sequences(token_train_x)
test_inputs = pad_sequences(token_test_x)
val_inputs = pad_sequences(token_val_x)